In [1]:
%load_ext autoreload
%autoreload 2

# Import necessary modules 
import numpy as np
import pandas as pd
from itertools import product
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import torch
from tqdm import tqdm

from rbf_volatility_surface import RBFVolatilitySurface
from smoothness_prior import RBFQuadraticSmoothnessPrior
from dataset_sabr import generate_sabr_call_options
from surface_vae_trainer import SurfaceVAETrainer
from dupire_pinn_trainer import DupirePINNTrainer
from evolutionary_algorithm import EvolutionaryAlgorithm

In [72]:
# Define the strike price list and maturity time list
strike_price_list = np.array([0.75, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3, 1.5])
maturity_time_list = np.array([0.02, 0.08, 0.17, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0])

# Create the product grid of maturity times and strike prices
product_grid = list(product(maturity_time_list, strike_price_list))
maturity_times, strike_prices = zip(*product_grid)

# Convert to arrays for further operations
maturity_times = np.array(maturity_times)
strike_prices = np.array(strike_prices)

# Variance formula for log-uniform distribution
def log_uniform_variance(a, b):
    log_term = np.log(b / a)
    var = ((b ** 2 - a ** 2) / (2 * log_term)) - ((b - a) / log_term) ** 2
    return var

# Calculate standard deviations for maturity times and strike prices
maturity_std = np.sqrt(log_uniform_variance(maturity_time_list.min(), maturity_time_list.max()))
strike_std = np.sqrt(log_uniform_variance(strike_price_list.min(), strike_price_list.max()))

# Define the SABR model parameters
alpha = 0.20  # Stochastic volatility parameter
beta = 0.50   # Elasticity parameter
rho = -0.75   # Correlation between asset price and volatility
nu = 1.0      # Volatility of volatility parameter

# Other model parameters
risk_free_rate = np.log(1.02)  # Risk-free interest rate
underlying_price = 1.0         # Current price of the underlying asset

# Generate the dataset using the SABR model and Black-Scholes formula
call_option_dataset = generate_sabr_call_options(
    alpha=alpha,
    beta=beta,
    rho=rho,
    nu=nu,
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    risk_free_rate=risk_free_rate,
    underlying_price=underlying_price
)

# Maturity times and strike prices from the previous product grid setup
hypothetical_maturity_time_list = np.logspace(np.log10(0.01), np.log10(3.1), 100)
hypothetical_strike_price_list = np.logspace(np.log10(0.7), np.log10(1.75), 100)

# Create the product grid of maturity times and strike prices
hypothetical_product_grid = list(product(hypothetical_maturity_time_list, hypothetical_strike_price_list))
hypothetical_maturity_times, hypothetical_strike_prices = zip(*hypothetical_product_grid)
hypothetical_maturity_times, hypothetical_strike_prices = np.array(hypothetical_maturity_times), np.array(hypothetical_strike_prices)

# Reshape the data for 3D surface plotting
hypothetical_maturities_grid = hypothetical_maturity_times.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))  
hypothetical_strikes_grid = hypothetical_strike_prices.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))

In [3]:
n_roots = 350
# n_roots = 10
smoothness_controller = 3.274549162877732e-05

# Initialize the RBFQuadraticSmoothnessPrior class
smoothness_prior = RBFQuadraticSmoothnessPrior(
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    maturity_std=maturity_std,
    strike_std=strike_std,
    n_roots=n_roots,
    smoothness_controller=smoothness_controller,
    random_state=0,
)

prior_covariance_matrix = smoothness_prior.prior_covariance()
prior_eigenvalues = np.sort(np.linalg.eigvalsh(prior_covariance_matrix))[::-1].copy()

# The constant_volatility is set to a reasonable value
constant_volatility = RBFVolatilitySurface.calculate_constant_volatility(
    call_option_dataset["Implied Volatility"],
    call_option_dataset["Time to Maturity"],
    call_option_dataset["Strike Price"],
    risk_free_rate,
    underlying_price
)

sampled_surface_coefficients = smoothness_prior.sample_smooth_surfaces(1000)

# Loop through the sampled coefficients 
sampled_volatilities = []
for coefficients in sampled_surface_coefficients:
    
    # Initialize the RBFVolatilitySurface class for each set of coefficients
    rbf_surface = RBFVolatilitySurface(
        coefficients=coefficients,
        maturity_times=maturity_times,
        strike_prices=strike_prices,
        maturity_std=maturity_std,
        strike_std=strike_std,
        constant_volatility=constant_volatility
    )

    # Generate the volatility surface over the product grid of times and strikes
    surface_volatilities = [
        rbf_surface.implied_volatility_surface(T, K)
        for T, K in product_grid
    ]
    sampled_volatilities.extend(surface_volatilities)

hidden_dim = 64
n_layers = 8
batch_size = 100
pde_loss_coefficient = 650.0
maturity_zero_loss_coefficient = 800.0
strike_zero_loss_coefficient = 40.0
strike_infinity_loss_coefficient = 200.0
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-4
pre_train_epochs = 200
fine_tune_epochs = 20
maturity_min = maturity_time_list.min()
maturity_max = maturity_time_list.max()
strike_min = strike_price_list.min()
strike_max = strike_price_list.max()
volatility_mean = np.mean(sampled_volatilities)
volatility_std = np.std(sampled_volatilities)
strike_infinity = 2.5
device = 'cpu'

# Initialize the DupirePINNTrainer class
torch.manual_seed(0)
pinn_trainer = DupirePINNTrainer(
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    batch_size=batch_size,
    pde_loss_coefficient=pde_loss_coefficient,
    maturity_zero_loss_coefficient=maturity_zero_loss_coefficient,
    strike_zero_loss_coefficient=strike_zero_loss_coefficient,
    strike_infinity_loss_coefficient=strike_infinity_loss_coefficient,
    pre_train_learning_rate=pre_train_learning_rate,
    fine_tune_learning_rate=fine_tune_learning_rate,
    pre_train_epochs=pre_train_epochs,
    fine_tune_epochs=fine_tune_epochs,
    maturity_min=maturity_min,
    maturity_max=maturity_max,
    strike_min=strike_min,
    strike_max=strike_max,
    volatility_mean=volatility_mean,
    volatility_std=volatility_std,
    maturity_time_list=maturity_time_list,
    strike_price_list=strike_price_list,
    strike_std=strike_std,
    maturity_std=maturity_std,
    constant_volatility=constant_volatility,
    strike_infinity=strike_infinity,
    device=device
)

pinn_trainer.load_model()

pinn_trainer.dupire_price_prediction_loss(
    torch.tensor(sampled_surface_coefficients, device=device, dtype=torch.float32),
    call_option_dataset["Call Option Price"],
    call_option_dataset["Time to Maturity"],
    call_option_dataset["Strike Price"],
)
print()    

PINN Model loaded from models/pinn_model.pth.



In [4]:
latent_dim = 40  # Latent dimension
data_dim = 100  # Data dimension of input
hidden_dim = 512
n_layers = 8
latent_diagonal = prior_eigenvalues[:latent_dim]  # Eigenvalues for latent prior
batch_size = 1000  # Batch size for training
beta_ = 1.0  # Beta value for beta-VAE
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-4  # Fine-tune learning rate
pre_train_epochs = 600  # Number of pre-train epochs
fine_tune_epochs = 20  # Number of fine-tune epochs
device = "cpu"  # Use CPU as the device

torch.manual_seed(2)
vae_trainer = SurfaceVAETrainer(
    latent_dim=latent_dim,
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    data_dim=data_dim,
    latent_diagonal=latent_diagonal,
    batch_size=batch_size,
    beta=beta_,
    pre_train_learning_rate=pre_train_learning_rate,
    fine_tune_learning_rate=fine_tune_learning_rate,
    pre_train_epochs=pre_train_epochs,
    fine_tune_epochs=fine_tune_epochs,
    device=device,
)

vae_trainer.load_model()

VAE Model loaded from models/vae_model.pth.


In [ ]:
# Define the parameter grid
population_sizes = [10, 100, 1000]
mutation_strengths = [0.1, 1.0, 10.0]
selection_pressure_parameters = [0.01, 0.1, 0.3, 0.5, 0.7]
n_generations = 250

# Store results for each configuration
results = []

for population_size, mutation_strength, selection_pressure_parameter in tqdm(
    product(population_sizes, mutation_strengths, selection_pressure_parameters),
    total=len(population_sizes) * len(mutation_strengths) * len(selection_pressure_parameters),
    desc="Processing combinations"
):
    # Initialize Evolutionary Algorithm with the current parameters
    torch.manual_seed(2)
    evolution_algo = EvolutionaryAlgorithm(
        vae_trainer=vae_trainer,
        pinn_trainer=pinn_trainer,
        latent_diagonal=latent_diagonal,
        population_size=population_size,
        mutation_strength=mutation_strength,
        selection_pressure_parameter=selection_pressure_parameter,
        n_generations=n_generations,  # You can adjust n_generations accordingly
    )

    # Run the optimization
    evolution_algo.optimize()

    # Metrics calculation
    final_best_fitness = evolution_algo.optimization_history["Best Fitness"][-1]  # Best fitness
    initial_fitness = evolution_algo.optimization_history["Best Fitness"][0]
    n_generation = n_generations

    # Calculate the diversity
    population_variance = np.var(evolution_algo.population.cpu().numpy(), axis=0)
    diversity = np.sum(population_variance / latent_diagonal)

    # Store the results for this configuration
    results.append({
        "population_size": population_size,
        "mutation_strength": mutation_strength,
        "selection_pressure_parameter": selection_pressure_parameter,
        "final_best_fitness": final_best_fitness,
        "diversity": diversity,
    })

In [6]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Rank the configurations based on each metric
results_df["rank_best_fitness"] = results_df["final_best_fitness"].rank(ascending=False)
results_df["rank_diversity"] = results_df["diversity"].rank(ascending=False)

# Calculate the average rank
results_df["average_rank"] = results_df[["rank_best_fitness", "rank_diversity"]].mean(axis=1)

# Sort by the average rank (lower is better)
results_df_sorted = results_df.sort_values(by="average_rank", ascending=True)

# Display the sorted results
results_df_sorted

,population_size,mutation_strength,selection_pressure_parameter,final_best_fitness,diversity,rank_best_fitness,rank_diversity,average_rank
44,1000,10.0,0.70,-0.183091,27315.933789,1.0,7.0,4.0
40,1000,10.0,0.01,-0.187259,31502.031498,7.0,1.0,4.0
41,1000,10.0,0.10,-0.188774,30733.788739,8.0,2.0,5.0
28,100,10.0,0.50,-0.184613,24397.205890,2.0,9.0,5.5
43,1000,10.0,0.50,-0.185050,26679.706559,3.0,8.0,5.5
42,1000,10.0,0.30,-0.185870,28844.631800,6.0,5.0,5.5
27,100,10.0,0.30,-0.190229,28890.646068,9.0,4.0,6.5
26,100,10.0,0.10,-0.191150,29510.554147,11.0,3.0,7.0
29,100,10.0,0.70,-0.185320,22416.674624,5.0,10.0,7.5
14,10,10.0,0.70,-0.185091,11894.744302,4.0,15.0,9.5


In [ ]:
# Define the parameter grid
population_sizes = [100]
mutation_strengths = [0.1, 1.0, 10.0, 100, 1000]
selection_pressure_parameters = [0.01, 0.1, 0.3, 0.5, 0.7]
n_generations = 500

# Store results for each configuration
results = []

for population_size, mutation_strength, selection_pressure_parameter in tqdm(
    product(population_sizes, mutation_strengths, selection_pressure_parameters),
    total=len(population_sizes) * len(mutation_strengths) * len(selection_pressure_parameters),
    desc="Processing combinations"
):
    # Initialize Evolutionary Algorithm with the current parameters
    torch.manual_seed(2)
    evolution_algo = EvolutionaryAlgorithm(
        vae_trainer=vae_trainer,
        pinn_trainer=pinn_trainer,
        latent_diagonal=latent_diagonal,
        population_size=population_size,
        mutation_strength=mutation_strength,
        selection_pressure_parameter=selection_pressure_parameter,
        n_generations=n_generations,  # You can adjust n_generations accordingly
    )

    # Run the optimization
    evolution_algo.optimize()

    # Metrics calculation
    final_best_fitness = evolution_algo.optimization_history["Best Fitness"][-1]  # Best fitness
    initial_fitness = evolution_algo.optimization_history["Best Fitness"][0]
    n_generation = n_generations

    # Calculate the diversity
    population_variance = np.var(evolution_algo.population.cpu().numpy(), axis=0)
    diversity = np.sum(population_variance / latent_diagonal)

    # Store the results for this configuration
    results.append({
        "population_size": population_size,
        "mutation_strength": mutation_strength,
        "selection_pressure_parameter": selection_pressure_parameter,
        "final_best_fitness": final_best_fitness,
        "diversity": diversity,
    })

In [8]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Rank the configurations based on each metric
results_df["rank_best_fitness"] = results_df["final_best_fitness"].rank(ascending=False)
results_df["rank_diversity"] = results_df["diversity"].rank(ascending=False)

# Calculate the average rank
results_df["average_rank"] = results_df[["rank_best_fitness", "rank_diversity"]].mean(axis=1)

# Sort by the average rank (lower is better)
results_df_sorted = results_df.sort_values(by="average_rank", ascending=True)

# Display the sorted results
results_df_sorted

,population_size,mutation_strength,selection_pressure_parameter,final_best_fitness,diversity,rank_best_fitness,rank_diversity,average_rank
24,100,1000.0,0.70,-0.189127,2.901874e+08,8.0,2.0,5.00
17,100,100.0,0.30,-0.186290,2.700472e+06,4.0,7.0,5.50
19,100,100.0,0.70,-0.183977,2.242630e+06,2.0,10.0,6.00
13,100,10.0,0.50,-0.183963,2.800802e+04,1.0,11.0,6.00
23,100,1000.0,0.50,-0.189349,2.765016e+08,9.0,3.0,6.00
18,100,100.0,0.50,-0.186349,2.431733e+06,5.0,9.0,7.00
22,100,1000.0,0.30,-0.193076,3.322783e+08,14.5,1.0,7.75
16,100,100.0,0.10,-0.189738,2.838208e+06,10.0,6.0,8.00
14,100,10.0,0.70,-0.184339,2.460161e+04,3.0,14.0,8.50
12,100,10.0,0.30,-0.187641,2.529852e+04,6.0,13.0,9.50


In [76]:
# Define evolutionary algorithm hyperparameters
population_size = 100 # Population size
mutation_strength = 100.0  # Mutation strength for Gaussian noise
selection_pressure_parameter = 0.7  # Selection pressure parameter (eta)
n_generations = 500  # Number of generations for the evolutionary process
torch.manual_seed(2)
# Initialize the EvolutionaryAlgorithm
evolution_algo = EvolutionaryAlgorithm(
    vae_trainer=vae_trainer,
    pinn_trainer=pinn_trainer,
    latent_diagonal=latent_diagonal,
    population_size=population_size,
    mutation_strength=mutation_strength,
    selection_pressure_parameter=selection_pressure_parameter,
    n_generations=n_generations,
)

# Run the optimization process
evolution_algo.optimize()

# Create the figure
fig = go.Figure()

# Calculate the diversity
population_variance = np.var(evolution_algo.population.cpu().numpy(), axis=0)
diversity = np.sum(population_variance / latent_diagonal)

optimization_history = pd.DataFrame(evolution_algo.optimization_history)
best_fitness = optimization_history["Best Fitness"].iloc[-1]

# Add trace for best fitness
fig.add_trace(go.Scatter(
    x=list(range(1, len(optimization_history["Best Fitness"]) + 1)),
    y=optimization_history["Best Fitness"],
    mode='lines+markers',
    name='Best Fitness',
    line=dict(color='royalblue', width=3),
    marker=dict(size=6, symbol='circle'),
    hovertemplate='<b>Iteration %{x}</b><br>Best Fitness: %{y:.4f}<extra></extra>'
))

# Add trace for average fitness
fig.add_trace(go.Scatter(
    x=list(range(1, len(optimization_history["Average Fitness"]) + 1)),
    y=optimization_history["Average Fitness"],
    mode='lines+markers',
    name='Average Fitness',
    line=dict(color='firebrick', width=3, dash='dash'),
    marker=dict(size=6, symbol='square'),
    hovertemplate='<b>Iteration %{x}</b><br>Average Fitness: %{y:.4f}<extra></extra>'
))

# Add trace for avg fitness - 1.96 * std fitness
fig.add_trace(go.Scatter(
    x=list(range(1, len(optimization_history["0.05 Percentile Fitness"]) + 1)),
    y=optimization_history["0.05 Percentile Fitness"],
    mode='lines+markers',
    name='0.05 Percentile Fitness',
    line=dict(color='green', width=3, dash='dot'),
    marker=dict(size=6, symbol='triangle-up'),
    hovertemplate='<b>Iteration %{x}</b><br>0.05 Percentile Fitness: %{y:.4f}<extra></extra>'
))

# Add an annotation for best fitness and diversity
fig.add_annotation(
    xref='paper', yref='paper', 
    x=0.96, y=0.05,
    text=f"Best Fitness: {best_fitness:.4f}<br>Diversity: {diversity:.4f}",
    showarrow=False,
    # font=dict(
    #     size=14,
    #     color="black"
    # ),
    align="left",
    # bordercolor="black",
    borderwidth=2,
    borderpad=4,
    bgcolor="white",
    opacity=1.
)

# Update layout to make the plot detailed and visually appealing
fig.update_layout(
    title="Initial Evolutionary Optimization History",
    xaxis_title="Generation",
    yaxis_title="Fitness Value",
    legend=dict(
        x=0.75,
        y=0.15,
    ),
    hovermode='x unified',
    width=900,
    height=900,
)

# Show the plot
fig.show()

100%|██████████| 500/500 [00:03<00:00, 150.11it/s]


In [77]:
fig.write_image('figs/initial_evolutionary_optimization_history.png', format='png', scale=4, width=900, height=900)

In [79]:
fine_tune_surface_coefficients = evolution_algo.final_population.values

In [ ]:
# Sample 9 surface coefficients using the smoothness prior
n_samples = 9
sampled_coefficients = fine_tune_surface_coefficients[:9]

# The constant_volatility is set to a reasonable value
constant_volatility = RBFVolatilitySurface.calculate_constant_volatility(
    call_option_dataset["Implied Volatility"],
    call_option_dataset["Time to Maturity"],
    call_option_dataset["Strike Price"],
    risk_free_rate,
    underlying_price
)

# Create the 3x3 subplots
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=[f"Sample Surface {i+1}" for i in range(n_samples)],
    specs=[[{'type': 'surface'}] * 3] * 3,
    vertical_spacing=0.05, horizontal_spacing=0.05
)

# Loop through the sampled coefficients to generate and plot each surface
for idx, coefficients in enumerate(sampled_coefficients):
    row = (idx // 3) + 1
    col = (idx % 3) + 1
    
    # Initialize the RBFVolatilitySurface class for each set of coefficients
    rbf_surface = RBFVolatilitySurface(
        coefficients=coefficients,
        maturity_times=maturity_times,
        strike_prices=strike_prices,
        maturity_std=maturity_std,
        strike_std=strike_std,
        constant_volatility=constant_volatility
    )

    # Generate the volatility surface over the product grid of times and strikes
    surface_volatilities = np.array([
        rbf_surface.implied_volatility_surface(T, K)
        for T, K in hypothetical_product_grid
    ]).reshape(len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list))

    # Add the surface plot to the subplot
    fig.add_trace(
        go.Surface(
            x=hypothetical_strikes_grid,
            y=hypothetical_maturities_grid,
            z=surface_volatilities,
            showscale=False  # Hide the scale on individual plots
        ),
        row=row, col=col
    )

for i in range(1, 4):  # Rows
    for j in range(1, 4):  # Columns
        fig.update_scenes(
            dict(
                xaxis_title="Strike Price",
                yaxis_title="Time to Maturity",
                zaxis_title="Implied Volatility",
                aspectratio=dict(x=1, y=1, z=0.8)
            ),
            row=i, col=j
        )    

# Update the layout of the figure
fig.update_layout(
    title="Top Volatility Surfaces from the Initial Evolutionary Optimization",
    height=1500, width=1500, title_x=0.5,
)

# Show the plot
fig.show()
fig.write_image('figs/samples_initial_evolutionary_optimization.png', format='png', scale=4, width=900, height=900)

In [94]:
hidden_dim = 64
n_layers = 8
batch_size = 100
pde_loss_coefficient = 650.0 * 1e-7
maturity_zero_loss_coefficient = 800.0
strike_zero_loss_coefficient = 40.0
strike_infinity_loss_coefficient = 200.0
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-2
pre_train_epochs = 200
fine_tune_epochs = 200
maturity_min = maturity_time_list.min()
maturity_max = maturity_time_list.max()
strike_min = strike_price_list.min()
strike_max = strike_price_list.max()
volatility_mean = np.mean(sampled_volatilities)
volatility_std = np.std(sampled_volatilities)
strike_infinity = 2.5
device = 'cpu'

# Initialize the DupirePINNTrainer class
pinn_trainer_ = DupirePINNTrainer(
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    batch_size=batch_size,
    pde_loss_coefficient=pde_loss_coefficient,
    maturity_zero_loss_coefficient=maturity_zero_loss_coefficient,
    strike_zero_loss_coefficient=strike_zero_loss_coefficient,
    strike_infinity_loss_coefficient=strike_infinity_loss_coefficient,
    pre_train_learning_rate=pre_train_learning_rate,
    fine_tune_learning_rate=fine_tune_learning_rate,
    pre_train_epochs=pre_train_epochs,
    fine_tune_epochs=fine_tune_epochs,
    maturity_min=maturity_min,
    maturity_max=maturity_max,
    strike_min=strike_min,
    strike_max=strike_max,
    volatility_mean=volatility_mean,
    volatility_std=volatility_std,
    maturity_time_list=maturity_time_list,
    strike_price_list=strike_price_list,
    strike_std=strike_std,
    maturity_std=maturity_std,
    constant_volatility=constant_volatility,
    strike_infinity=strike_infinity,
    device=device
)

pinn_trainer_.load_model()

pinn_trainer_.fine_tune(
    sampled_surface_coefficients=fine_tune_surface_coefficients,
)

loss_history = pd.DataFrame(pinn_trainer_.fine_tune_loss_history)

# Create a subplot figure with 2x2 grid for individual losses, and a third row spanning the entire width for total loss
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=("PDE Loss", "Zero Maturity Loss", "Zero Strike Loss", "Infinity Strike Loss", "Total Loss"),
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'scatter'}, {'type': 'scatter'}],
           [{'colspan': 2, 'type': 'scatter'}, None]],
    vertical_spacing=0.1,
    horizontal_spacing=0.1
)

# Add traces for individual losses
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["PDE Loss"], mode="lines", name="PDE Loss"), row=1, col=1)
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["Zero Maturity Loss"], mode="lines", name="Zero Maturity Loss"), row=1, col=2)
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["Zero Strike Loss"], mode="lines", name="Zero Strike Loss"), row=2, col=1)
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["Infinity Strike Loss"], mode="lines", name="Infinity Strike Loss"), row=2, col=2)

# Add a trace for the total loss spanning the entire third row
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["Total Loss"], mode="lines", name="Total Loss"), row=3, col=1)

# Update the layout to include 'Iterations' as the x-axis name for each subplot
fig.update_xaxes(title_text="Iterations", row=1, col=1)
fig.update_xaxes(title_text="Iterations", row=1, col=2)
fig.update_xaxes(title_text="Iterations", row=2, col=1)
fig.update_xaxes(title_text="Iterations", row=2, col=2)
fig.update_xaxes(title_text="Iterations", row=3, col=1)  # The third row spans two columns

# Update the layout
fig.update_layout(height=900, width=900, title_text="PINN Training Losses", showlegend=False)

# Show the plot
fig.show()

PINN Model loaded from models/pinn_model.pth.


  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:43<00:00,  4.59it/s]


In [86]:
latent_dim = 40  # Latent dimension
data_dim = 100  # Data dimension of input
hidden_dim = 512
n_layers = 8
latent_diagonal = prior_eigenvalues[:latent_dim]  # Eigenvalues for latent prior
batch_size = 1000  # Batch size for training
beta_ = 1.0  # Beta value for beta-VAE
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-7  # Fine-tune learning rate
pre_train_epochs = 600  # Number of pre-train epochs
fine_tune_epochs = 2  # Number of fine-tune epochs
device = "cpu"  # Use CPU as the device

torch.manual_seed(2)
vae_trainer_ = SurfaceVAETrainer(
    latent_dim=latent_dim,
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    data_dim=data_dim,
    latent_diagonal=latent_diagonal,
    batch_size=batch_size,
    beta=beta_,
    pre_train_learning_rate=pre_train_learning_rate,
    fine_tune_learning_rate=fine_tune_learning_rate,
    pre_train_epochs=pre_train_epochs,
    fine_tune_epochs=fine_tune_epochs,
    device=device,
)

vae_trainer_.load_model()

# Train the model using pre_train
vae_trainer_.fine_tune(
    sampled_surface_coefficients=fine_tune_surface_coefficients,
)

loss_history = pd.DataFrame(vae_trainer_.fine_tune_loss_history)

# Create a subplot figure with 1x2 grid for individual losses, and a second row spanning the entire width for total loss
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Reconstruction Loss", "KL Loss", "Total Loss"),
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'colspan': 2, 'type': 'scatter'}, None]],
    vertical_spacing=0.1,
    horizontal_spacing=0.1
)

# Add traces for individual losses
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["Reconstruction Loss"], mode="lines", name="Reconstruction Loss"), row=1, col=1)
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["KL Loss"], mode="lines", name="KL Loss"), row=1, col=2)

# Add a trace for the total loss spanning the entire second row
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["Total Loss"], mode="lines", name="Total Loss"), row=2, col=1)

# Update the layout to include 'Iterations' as the x-axis name for each subplot
fig.update_xaxes(title_text="Iterations", row=1, col=1)
fig.update_xaxes(title_text="Iterations", row=1, col=2)
fig.update_xaxes(title_text="Iterations", row=2, col=1)  # The third row spans two columns

fig.update_yaxes(type="log", row=1, col=1)
fig.update_yaxes(type="log", row=1, col=2)
fig.update_yaxes(type="log", row=2, col=1)  # The third row spans two columns

# Update the layout
fig.update_layout(height=900, width=900, title_text="Beta-VAE Training Losses", showlegend=False)

# Show the plot
fig.show()

VAE Model loaded from models/vae_model.pth.


100%|██████████| 2/2 [00:00<00:00, 47.94it/s]
